In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#CSV of OHLC information from Dukascopy
df = pd.read_csv("GBPUSD_Candlestick_1_Hour_BID_01.01.2010-01.01.2021.csv")
#Options to replace CSV
#"USA30.IDXUSD_Candlestick_1_Hour_BID_01.01.2020-09.04.2022.csv")
#"GBPUSD_Candlestick_1_Hour_BID_04.01.2015-09.04.2022.csv"
#"GBPUSD_Candlestick_1_Hour_BID_01.01.2022-09.04.2022.csv")
#"GBPUSD_Candlestick_1_Hour_BID_01.01.2010-01.01.2021.csv")

In [3]:
#Some data fitting that will be automated in the future...
df = df.iloc[:-24]
df['datetime'] = "NA"
df['datetime'] = pd.date_range(start='01/01/2010', end='01/01/2021', freq='1H')#start='01/01/2010', end='01/02/2021', freq='1H')
df['weekday'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df['week'] = df['datetime'].dt.week
df.set_index('datetime', inplace=True)

#this doesnt mimic conventional charts, would need to shift starting point (Or just use data in NY time)
df['WeekNum'] = np.divmod(np.arange(len(df)),168)[0]+1

/tmp/ipykernel_6685/2929112884.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['datetime'].dt.week


In [4]:
#filter out the NaN values and anything that is 0 volume (Weekends & Holidays)
indexZeroV = df[ df['Volume'] == 0].index
df.drop(indexZeroV, inplace = True)

#create df for bull weeks and bear weeks
#group by week number
weekly_grp = df.groupby('WeekNum')
dfW = pd.DataFrame()

dfW['wHigh'] = df.groupby('WeekNum')['High'].max()
dfW['wLow'] = df.groupby('WeekNum')['Low'].min()
dfW['wOpen'] = df.groupby('WeekNum')['Open'].first()
dfW['wClose'] = df.groupby('WeekNum')['Close'].last()

df.isna().sum()

Localtime    0
Open         0
High         0
Low          0
Close        0
Volume       0
weekday      0
hour         0
week         0
WeekNum      0
dtype: int64

In [5]:
#calculate the outcome of the week (its open minus its close)
dfW['OC'] = dfW['wOpen'] - dfW['wClose']
#take OC of each weeknum then place it in row with that weeknum in df
bins = [np.NINF,0,np.Inf]
state_groups = ["Bull","Bear"]
dfW['Outcome'] = pd.cut(dfW.OC,bins,labels = state_groups)
#duplicate rows
dff = pd.DataFrame()
dff = dfW.loc[dfW.index.repeat(168)]
#move Outcome column to original df
df_dropped_last_n = dff.iloc[:-143]
df_dropped_last_n = df_dropped_last_n.reset_index()
#del df['Localtime']
del df_dropped_last_n['WeekNum']
df = df.reset_index().join(df_dropped_last_n)
df.set_index('datetime', inplace=True)


In [6]:
#We want to join highs of the week for bear outcomes and lows of the week datetime ids for bull outcomes
weekly_grp_bull = df.groupby('WeekNum')['Low'].idxmin()
weekly_grp_bear = df.groupby('WeekNum')['High'].idxmax()

dfidxmin = pd.DataFrame()
dfidxmin = weekly_grp_bull.loc[weekly_grp_bull.index.repeat(168)]

#move Outcome column to original df
dfidx_min_dropped = dfidxmin.iloc[:-143]
dfidx_min_dropped = dfidx_min_dropped.reset_index()

dfidxmax = pd.DataFrame()
dfidxmax = weekly_grp_bear.loc[weekly_grp_bear.index.repeat(168)]

#move Outcome column to original df
dfidx_max_dropped = dfidxmax.iloc[:-143]
dfidx_max_dropped = dfidx_max_dropped.reset_index()

#merge idx for highs and lows of the week...
df = df.reset_index()
df['WH_ID'] = dfidx_max_dropped['High']
df['WL_ID'] = dfidx_min_dropped['Low']
df.set_index('datetime', inplace=True)

#Classify each row as datetime appriate for the outcome week
#Now we are gonna group by each week and then group that group by bull and bear outcome weeks
bins_classify = ["Bull","Bear"]
states_datetime = [np.NINF,0,np.Inf]

#dfW['Outcome'] = pd.cut(dfW.OC,bins,labels = state_groups)


#df_grped = df.groupby(['WeekNum'], as_index = False)
#After that we will run statistics regarding what day of the week is HOW/LOW what % of the time...


In [7]:
#Create the chosen row that gives the ID of the corresponding high/low of week
#df['Chosen'] = "NA"
df.loc[df.Outcome == "Bear", 'Chosen'] = df['WH_ID']
df.loc[df.Outcome == "Bull", 'Chosen'] = df['WL_ID']
#df['ChosenOpp'] = "NA"
df.loc[df.Outcome == "Bear", 'ChosenOpp'] = df['WL_ID']
df.loc[df.Outcome == "Bull", 'ChosenOpp'] = df['WH_ID']

#Show the DF
df

,Localtime,Open,High,Low,Close,Volume,weekday,hour,week,WeekNum,wHigh,wLow,wOpen,wClose,OC,Outcome,WH_ID,WL_ID,Chosen,ChosenOpp
datetime,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,01.01.2010 00:00:00.000 GMT-0600,1.61673,1.61696,1.61613,1.61678,9958.2002,Friday,0,53,1,1.62415,1.58960,1.61673,1.59350,0.02323,Bear,2010-01-04 04:00:00,2010-01-07 06:00:00,2010-01-04 04:00:00,2010-01-07 06:00:00
2010-01-01 01:00:00,01.01.2010 01:00:00.000 GMT-0600,1.61672,1.61686,1.61603,1.61672,16954.3008,Friday,1,53,1,1.62415,1.58960,1.61673,1.59350,0.02323,Bear,2010-01-04 04:00:00,2010-01-07 06:00:00,2010-01-04 04:00:00,2010-01-07 06:00:00
2010-01-01 02:00:00,01.01.2010 02:00:00.000 GMT-0600,1.61672,1.61693,1.61590,1.61670,12790.5000,Friday,2,53,1,1.62415,1.58960,1.61673,1.59350,0.02323,Bear,2010-01-04 04:00:00,2010-01-07 06:00:00,2010-01-04 04:00:00,2010-01-07 06:00:00
2010-01-01 03:00:00,01.01.2010 03:00:00.000 GMT-0600,1.61662,1.61678,1.61582,1.61667,13053.0996,Friday,3,53,1,1.62415,1.58960,1.61673,1.59350,0.02323,Bear,2010-01-04 04:00:00,2010-01-07 06:00:00,2010-01-04 04:00:00,2010-01-07 06:00:00
2010-01-01 04:00:00,01.01.2010 04:00:00.000 GMT-0600,1.61667,1.61678,1.61598,1.61670,13227.4004,Friday,4,53,1,1.62415,1.58960,1.61673,1.59350,0.02323,Bear,2010-01-04 04:00:00,2010-01-07 06:00:00,2010-01-04 04:00:00,2010-01-07 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 11:00:00,31.12.2020 11:00:00.000 GMT-0600,1.36630,1.36783,1.36493,1.36501,10372.9297,Thursday,11,53,574,1.33206,1.30425,1.31339,1.30724,0.00615,Bear,2017-11-01 03:00:00,2017-11-02 11:00:00,2017-11-01 03:00:00,2017-11-02 11:00:00
2020-12-31 12:00:00,31.12.2020 12:00:00.000 GMT-0600,1.36498,1.36619,1.36498,1.36584,5548.8799,Thursday,12,53,574,1.33206,1.30425,1.31339,1.30724,0.00615,Bear,2017-11-01 03:00:00,2017-11-02 11:00:00,2017-11-01 03:00:00,2017-11-02 11:00:00
2020-12-31 13:00:00,31.12.2020 13:00:00.000 GMT-0600,1.36584,1.36748,1.36529,1.36669,4482.8999,Thursday,13,53,574,1.33206,1.30425,1.31339,1.30724,0.00615,Bear,2017-11-01 03:00:00,2017-11-02 11:00:00,2017-11-01 03:00:00,2017-11-02 11:00:00


In [8]:
#Look up the ID given from the Chosen column and get its day of the week, make that swing day col
#df['swingDOW'] = df.loc[df['Chosen']]
#df['swingDOW'] = 
moo = pd.DataFrame()
temp = pd.DataFrame()
hr = pd.DataFrame()


moo['ans'] = df.loc[df['Chosen']]['weekday']
temp['ansOpp'] = df.loc[df['ChosenOpp']]['weekday']
hr['myHR'] = df.loc[df['Chosen']]['hour']


moo = moo.reset_index()
del moo['datetime']

temp = temp.reset_index()
del temp['datetime']

hr = hr.reset_index()
del hr['datetime']

df = df.reset_index().join(moo)
df = df.reset_index().join(temp)
df = df.reset_index().join(hr)

del df['index']
df.set_index('datetime', inplace=True)


In [9]:
#Statistics associated with the ans column
df = df.drop(index=df[df['weekday'] == 'Saturday'].index) #doesnt work?


In [10]:
#Calculate the % where the end opening of the week meets its extremity
df['ans'].value_counts(normalize=True)

Friday       0.526515
Monday       0.161628
Tuesday      0.149383
Wednesday    0.076762
Thursday     0.044080
Sunday       0.041631
Name: ans, dtype: float64

In [11]:
#Calculate the % where the opposite end of the week meets its extremity
df['ansOpp'].value_counts(normalize=True)

Thursday     0.556748
Wednesday    0.222851
Tuesday      0.132241
Monday       0.053876
Friday       0.026938
Sunday       0.007347
Name: ansOpp, dtype: float64

In [12]:
#send to new CSV
#df.to_csv('file_done_newx.csv')
#verify 
df['myHR'].value_counts(normalize=True)

2     0.129792
0     0.119997
1     0.102854
3     0.096353
9     0.080814
8     0.063672
6     0.061223
7     0.061223
4     0.058774
5     0.053876
10    0.026938
15    0.019591
11    0.017142
12    0.017142
14    0.017142
17    0.014693
18    0.012245
23    0.012245
13    0.009796
19    0.007347
21    0.007347
20    0.007347
22    0.002449
Name: myHR, dtype: float64